In [1]:
import pandas as pd
import numpy as np
import datetime

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score

In [2]:
now = datetime.datetime.now()
date_csv = "{}{:02d}{:02d}".format(now.year,now.month,now.day)
df = pd.read_csv('dataset_{}.csv'.format(date_csv))
df_champions = pd.read_csv('dataset-champions.csv')
df

,bluetop,bluejungle,bluemid,blueadc,bluesupport,redtop,redjungle,redmid,redadc,redsupport,result
0,Mordekaiser,Graves,Zoe,Miss Fortune,Galio,Ornn,Kindred,Orianna,Ezreal,Leona,0
1,Gragas,Graves,Rumble,Kai'Sa,Alistar,Ornn,Olaf,Syndra,Miss Fortune,Galio,1
2,Gragas,Graves,Zoe,Yasuo,Alistar,Rumble,Nidalee,Yone,Miss Fortune,Galio,1
3,Karma,Pantheon,Syndra,Samira,Leona,Aatrox,Nidalee,Viktor,Kai'Sa,Galio,1
4,Jax,Hecarim,Orianna,Samira,Alistar,Camille,Lillia,Viktor,Kai'Sa,Maokai,1
...,...,...,...,...,...,...,...,...,...,...,...
5276,Renekton,Lillia,Ryze,Jinx,Tahm Kench,Urgot,Olaf,Orianna,Kai'Sa,Leona,0
5277,Ornn,Udyr,Viktor,Jinx,Tahm Kench,Darius,Hecarim,Twisted Fate,Aphelios,Thresh,0
5278,Aatrox,Volibear,Orianna,Jinx,Tahm Kench,Olaf,Hecarim,Corki,Aphelios,Thresh,1
5279,Sion,Udyr,Akali,Jinx,Leona,Renekton,Lillia,Orianna,Lucian,Braum,0


In [3]:
y = df['result'].copy()
X = df.drop('result',axis=1)
X

,bluetop,bluejungle,bluemid,blueadc,bluesupport,redtop,redjungle,redmid,redadc,redsupport
0,Mordekaiser,Graves,Zoe,Miss Fortune,Galio,Ornn,Kindred,Orianna,Ezreal,Leona
1,Gragas,Graves,Rumble,Kai'Sa,Alistar,Ornn,Olaf,Syndra,Miss Fortune,Galio
2,Gragas,Graves,Zoe,Yasuo,Alistar,Rumble,Nidalee,Yone,Miss Fortune,Galio
3,Karma,Pantheon,Syndra,Samira,Leona,Aatrox,Nidalee,Viktor,Kai'Sa,Galio
4,Jax,Hecarim,Orianna,Samira,Alistar,Camille,Lillia,Viktor,Kai'Sa,Maokai
...,...,...,...,...,...,...,...,...,...,...
5276,Renekton,Lillia,Ryze,Jinx,Tahm Kench,Urgot,Olaf,Orianna,Kai'Sa,Leona
5277,Ornn,Udyr,Viktor,Jinx,Tahm Kench,Darius,Hecarim,Twisted Fate,Aphelios,Thresh
5278,Aatrox,Volibear,Orianna,Jinx,Tahm Kench,Olaf,Hecarim,Corki,Aphelios,Thresh
5279,Sion,Udyr,Akali,Jinx,Leona,Renekton,Lillia,Orianna,Lucian,Braum


In [4]:
best_features_list = ['bluetop','bluejungle', 'bluemid', 'blueadc', 'bluesupport', 'redtop', 'redjungle', 'redmid', 'redadc', 'redsupport']
new_X = X[best_features_list]
for index, row in new_X.iterrows():
    row['bluetop'] = int(df_champions.loc[df_champions['id'] == row['bluetop']]['key'])
    row['bluejungle'] = int(df_champions.loc[df_champions['id'] == row['bluejungle']]['key'])
    row['bluemid'] = int(df_champions.loc[df_champions['id'] == row['bluemid']]['key'])    
    row['blueadc'] = int(df_champions.loc[df_champions['id'] == row['blueadc']]['key'])
    row['bluesupport'] = int(df_champions.loc[df_champions['id'] == row['bluesupport']]['key'])

    row['redtop'] = int(df_champions.loc[df_champions['id'] == row['redtop']]['key'])
    row['redjungle'] = int(df_champions.loc[df_champions['id'] == row['redjungle']]['key'])
    row['redmid'] = int(df_champions.loc[df_champions['id'] == row['redmid']]['key'])
    row['redadc'] = int(df_champions.loc[df_champions['id'] == row['redadc']]['key'])
    row['redsupport'] = int(df_champions.loc[df_champions['id'] == row['redsupport']]['key'])


blue_team = new_X[['bluetop','bluejungle', 'bluemid', 'blueadc', 'bluesupport']].values
red_team = new_X[['redtop', 'redjungle', 'redmid', 'redadc', 'redsupport']].values

a
Xayah
Tristana
Jinx
Jhin
Caitlyn
Ziggs
Gragas
Kai'Sa
Kai'Sa
Ezreal
Samira
Sivir
Senna
Senna
Senna
Senna
Kai'Sa
Xayah
Jhin
Kai'Sa
Aphelios
Samira
Samira
Kalista
Aphelios
Kai'Sa
Samira
Jhin
Kai'Sa
Xayah
Sivir
Kai'Sa
Senna
Kai'Sa
Jhin
Senna
Vayne
Samira
Xayah
Kai'Sa
Jhin
Senna
Aphelios
Aphelios
Samira
Aphelios
Jhin
Yasuo
Kai'Sa
Xayah
Miss Fortune
Senna
Kalista
Ezreal
Kai'Sa
Kai'Sa
Jhin
Samira
Ezreal
Xayah
Ezreal
Miss Fortune
Senna
Xayah
Kai'Sa
Senna
Xayah
Senna
Jinx
Xayah
Jinx
Kai'Sa
Maokai
Jinx
Ezreal
Vayne
Kai'Sa
Kai'Sa
Miss Fortune
Kai'Sa
Kai'Sa
Aphelios
Xayah
Kai'Sa
Miss Fortune
Kai'Sa
Aphelios
Kai'Sa
Ashe
Kai'Sa
Samira
Kalista
Aphelios
Xayah
Samira
Kai'Sa
Kai'Sa
Kai'Sa
Aphelios
Aphelios
Xayah
Xayah
Kai'Sa
Xayah
Aphelios
Jhin
Jhin
Jhin
Aphelios
Kai'Sa
Ashe
Aphelios
Kai'Sa
Aphelios
Senna
Kai'Sa
Samira
Tristana
Jhin
Aphelios
Ezreal
Sivir
Kai'Sa
Xayah
Kai'Sa
Ezreal
Ashe
Jhin
Miss Fortune
Samira
Kai'Sa
Jhin
Kai'Sa
Kai'Sa
Jhin
Samira
Varus
Tristana
Samira
Aphelios
Samira
Xayah
Jhin
Ezrea

In [6]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(new_X,y)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}

for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained
Support Vector Machine (Linear Kernel) trained
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [8]:
scores_list = []
for name,model in models.items():
    scores_list.append({
    'Model': name,
    'Accuracy': accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [9]:
scores

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,0.546831,0.512195,0.130705,0.208264
1,Support Vector Machine (Linear Kernel),0.547777,0.516129,0.132780,0.211221
2,Support Vector Machine (RBF Kernel),0.521287,0.450000,0.224066,0.299169
3,Decission Tree,0.531693,0.486022,0.468880,0.477297
4,Adaboost,0.548723,0.506849,0.383817,0.436836
5,Random Forest,0.531693,0.483871,0.404564,0.440678
6,Gradient Boosting Classifier,0.559130,0.523256,0.373444,0.435835
